In [32]:
import os, sys

import xarray as xr
import pandas as pd
import numpy as np

In [33]:
sys.path.append(os.path.join(os.getcwd(), 'Documents', 'mjo_and_rainfall_trend'))
import phase_calc_functions as phase_calc

In [132]:
reps: int = 4
active_phase_data = np.tile(np.arange(1, 9), reps)
inactive_phase_data = active_phase_data
phase_data = np.concatenate([active_phase_data, inactive_phase_data])
amplitude_data = [5] * len(inactive_phase_data) + [0] * len(inactive_phase_data)#np.tile(np.concatenate([[0], [5]*8]), reps)

In [133]:
len(phase_data), len(amplitude_data)

(64, 64)

In [134]:
phase_data

array([1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6,
       7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4,
       5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8])

In [135]:
time_range = pd.date_range(start='1974-01-01', freq='D', periods=len(phase_data))
time_range

DatetimeIndex(['1974-01-01', '1974-01-02', '1974-01-03', '1974-01-04',
               '1974-01-05', '1974-01-06', '1974-01-07', '1974-01-08',
               '1974-01-09', '1974-01-10', '1974-01-11', '1974-01-12',
               '1974-01-13', '1974-01-14', '1974-01-15', '1974-01-16',
               '1974-01-17', '1974-01-18', '1974-01-19', '1974-01-20',
               '1974-01-21', '1974-01-22', '1974-01-23', '1974-01-24',
               '1974-01-25', '1974-01-26', '1974-01-27', '1974-01-28',
               '1974-01-29', '1974-01-30', '1974-01-31', '1974-02-01',
               '1974-02-02', '1974-02-03', '1974-02-04', '1974-02-05',
               '1974-02-06', '1974-02-07', '1974-02-08', '1974-02-09',
               '1974-02-10', '1974-02-11', '1974-02-12', '1974-02-13',
               '1974-02-14', '1974-02-15', '1974-02-16', '1974-02-17',
               '1974-02-18', '1974-02-19', '1974-02-20', '1974-02-21',
               '1974-02-22', '1974-02-23', '1974-02-24', '1974-02-25',
      

In [136]:
# Create the mock xarray dataset
mock_rmm = xr.Dataset({
    'phase': xr.DataArray(phase_data, dims=['time'], coords={'time': time_range}),
    'amplitude': xr.DataArray(amplitude_data, dims=['time'], coords={'time': time_range})
})


In [137]:
mock_rmm

<xarray.Dataset>
Dimensions:    (time: 64)
Coordinates:
  * time       (time) datetime64[ns] 1974-01-01 1974-01-02 ... 1974-03-05
Data variables:
    phase      (time) int64 1 2 3 4 5 6 7 8 1 2 3 4 ... 5 6 7 8 1 2 3 4 5 6 7 8
    amplitude  (time) int64 5 5 5 5 5 5 5 5 5 5 5 5 ... 0 0 0 0 0 0 0 0 0 0 0 0

In [146]:
mock_awap = xr.DataArray(
            data=np.concatenate([active_phase_data, [0] * len(inactive_phase_data)]),
            coords={'time': time_range},
            dims=['time']
        )
mock_awap.name = 'precip'

In [147]:
mock_awap

<xarray.DataArray 'precip' (time: 64)>
array([1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6,
       7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Coordinates:
  * time     (time) datetime64[ns] 1974-01-01 1974-01-02 ... 1974-03-05

In [158]:
mock_awap['phase'] = ('time', phase_data)
mock_awap['amplitude'] = ('time', amplitude_data)

mock_awap

#mock_awap.where(mock_awap.phase==1, drop=True)

<xarray.DataArray 'precip' (time: 64)>
array([1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6,
       7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Coordinates:
  * time       (time) datetime64[ns] 1974-01-01 1974-01-02 ... 1974-03-05
    phase      (time) int64 1 2 3 4 5 6 7 8 1 2 3 4 ... 5 6 7 8 1 2 3 4 5 6 7 8
    amplitude  (time) int64 5 5 5 5 5 5 5 5 5 5 5 5 ... 0 0 0 0 0 0 0 0 0 0 0 0

In [149]:
result = phase_calc.split_into_1to8(mock_awap, mock_rmm)

In [155]:
result

<xarray.DataArray 'precip' (phase: 9, time: 64)>
array([[ 1., nan, nan, nan, nan, nan, nan, nan,  1., nan, nan, nan, nan,
        nan, nan, nan,  1., nan, nan, nan, nan, nan, nan, nan,  1., nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan,  2., nan, nan, nan, nan, nan, nan, nan,  2., nan, nan, nan,
        nan, nan, nan, nan,  2., nan, nan, nan, nan, nan, nan, nan,  2.,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan,  3., nan, nan, nan, nan, nan, nan, nan,  3., nan, nan,
        nan, nan, nan, nan, nan,  3., nan, nan, nan, nan, nan, nan, nan,
         3., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan,  4., nan, nan, nan, nan, nan, nan, nan,  4., nan,
        nan, nan, nan, nan, nan, nan,  4., nan, nan, nan, nan, nan, nan,
        nan,  4., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
...
       [nan, nan, nan, nan, nan,  6., nan, nan, nan, nan, nan, nan, nan,
         6., nan, nan, nan, nan, nan, nan, nan,  6., nan, nan, nan, nan,
        nan, nan, nan,  6., nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan,  7., nan, nan, nan, nan, nan, nan,
        nan,  7., nan, nan, nan, nan, nan, nan, nan,  7., nan, nan, nan,
        nan, nan, nan, nan,  7., nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan,  8., nan, nan, nan, nan, nan,
        nan, nan,  8., nan, nan, nan, nan, nan, nan, nan,  8., nan, nan,
        nan, nan, nan, nan, nan,  8., nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])
Coordinates:
  * time     (time) datetime64[ns] 1974-01-01 1974-01-02 ... 1974-03-05
  * phase    (phase) int64 1 2 3 4 5 6 7 8 0

In [153]:
assert all(result.sel(phase=1).dropna(dim='time').values == 1)
assert all(result.sel(phase=2).dropna(dim='time').values == 2)
assert all(result.sel(phase=3).dropna(dim='time').values == 3)
assert all(result.sel(phase=4).dropna(dim='time').values == 4)
assert all(result.sel(phase=5).dropna(dim='time').values == 5)
assert all(result.sel(phase=6).dropna(dim='time').values == 6)
assert all(result.sel(phase=7).dropna(dim='time').values == 7)
assert all(result.sel(phase=8).dropna(dim='time').values == 8)
assert all(result.sel(phase=0).dropna(dim='time').values == 0)